In [1]:
from PIL import Image
import os
import numpy as np
import random
node_saver = [] # 用来存分叉节点的信息，便于后边合并
leaf_saver = [] # 用来存叶子结点的信息

def Dec2Bi(number: int): #将10进制转换为2进制
    outcome = []
    for i in range(8):
        outcome.append(number%2)
        number = int(number/2)
    return outcome[-1::-1]

class tnode:# 树节点的定义
    def __init__(self, leaf = True, layer = None):
        self.counter = 0
        self.redSum = 0
        self.greenSum = 0
        self.blueSum = 0
        self.leaf = leaf
        self.child_node = []
        self.child_num = 0
        self.layer = layer
        
    def update(self,red,green,blue):
        self.redSum+=red
        self.greenSum+=green
        self.blueSum+=blue 

def born(parent_node: tnode):
    parent_node.child_node = [None for i in range(8)]
    parent_node.leaf = False
    leaf_saver.remove(parent_node)
    
class Tree:
    def __init__(self):
        self.root = tnode()
        self.root.layer = 1
        self.leaf_number = 0
        leaf_saver.append(self.root)
        born(self.root)

Octree = Tree() # 我们可爱的树

def insert(Tree,node,pixel,index):
    if(index >= 8):
        return True
    else:
        if(index == 0):
            node.update(pixel[0],pixel[1],pixel[2])
        red = Dec2Bi(pixel[0])[index]
        green = Dec2Bi(pixel[1])[index]
        blue = Dec2Bi(pixel[2])[index]
        loc = red*4+green*2+blue
        if(node.leaf):
            born(node)
            Tree.leaf_number = Tree.leaf_number - 1
        if(node.child_node[loc] == None):
            node.child_node[loc] = tnode()
            leaf_saver.append(node.child_node[loc])
            node.child_node[loc].layer = node.layer+1
            node.child_num += 1
            Tree.leaf_number += 1
            if(node.child_num == 2):
                node_saver.append((index+1,pixel,node.counter))
        node.child_node[loc].update(pixel[0],pixel[1],pixel[2])
        node.counter += 1
        insert(Tree,node.child_node[loc],pixel,index+1)
        
def deep_node(Tree):#返回最深的且节点数最小的分支的节点对象
    deepest = -1
    counter_min = 20000
    trace = []
    for branch in node_saver:
        if(branch[0]>deepest):
            deepest = branch[0]
    for branch in node_saver:
        if(branch[0]==deepest and branch[2]<counter_min):
            counter_min = branch[2]
    for branch in node_saver:
        if(branch[0]==deepest and branch[2]==counter_min):
            index = branch[0]
            pixel = branch[1]
            for i in range(index-1):
                red = Dec2Bi(pixel[0])[i]
                green = Dec2Bi(pixel[1])[i]
                blue = Dec2Bi(pixel[2])[i]
                loc = red*4+green*2+blue
                trace.append(loc)
            node_saver.remove(branch)
            break
    node = Tree.root
    for loc in trace:
        node = node.child_node[loc]
    return node

def merge(Tree):#合并这颗树的节点
    node = deep_node(Tree)
    for i in range(8):
        if(node.child_node[i]!=None):
            leaf_node = node.child_node[i]
            Tree.leaf_number = Tree.leaf_number - 1
            while(leaf_node.leaf == False): # 如果该节点不是叶子结点的情况
                for j in range(len(leaf_node.child_node)):
                    if(leaf_node.child_node[j]):
                        leaf_node = leaf_node.child_node[j]
                        break
            leaf_saver.remove(leaf_node)
            node.child_node[i]=None 
    node.leaf = True
    leaf_saver.append(node)
    node.child_num = 0
    Tree.leaf_number += 1

def start_merge(image):
    img = Image.open(image)
    data = np.array(img)
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            insert(Octree,Octree.root,data[i][j].tolist(),0)
    print('Tree construction complete!')
    Total = Octree.leaf_number
    while(Octree.leaf_number>256):
        if(Octree.leaf_number % 1000 == 0):
            print('{} has finished: {}/{}'.format(Octree.leaf_number/Total,Octree.leaf_number,Total))
        merge(Octree)

In [2]:
Octree.root.counter

0

In [3]:
start_merge("task2.jpg")

Tree construction complete!
0.979587838416986 has finished: 240000/245001
0.9673429904367737 has finished: 237000/245001
0.9591797584499655 has finished: 235000/245001
0.9510165264631573 has finished: 233000/245001
0.9469349104697532 has finished: 232000/245001
0.942853294476349 has finished: 231000/245001
0.9306084464961367 has finished: 228000/245001
0.9265268305027327 has finished: 227000/245001
0.9224452145093285 has finished: 226000/245001
0.9183635985159244 has finished: 225000/245001
0.9102003665291162 has finished: 223000/245001
0.9020371345423079 has finished: 221000/245001
0.8979555185489039 has finished: 220000/245001
0.8938739025554998 has finished: 219000/245001
0.8897922865620956 has finished: 218000/245001
0.8857106705686916 has finished: 217000/245001
0.8816290545752874 has finished: 216000/245001
0.8693842065950751 has finished: 213000/245001
0.865302590601671 has finished: 212000/245001
0.8612209746082669 has finished: 211000/245001
0.8489761266280546 has finished: 20

In [511]:
data2.shape

(768, 1365, 3)

In [512]:
768*1365

1048320

In [23]:
leaf_saver[1].counter

0

In [401]:
merge(Octree)

1
0


In [395]:
pixel = [255,255,255]

In [396]:
insert(Octree,Octree.root,pixel,0)

In [4]:
Octree.root.counter

1048320

In [5]:
Octree.leaf_number

253

In [28]:
txtName = "task2.txt"
f=open(txtName, "a+")
for i in range(253):
    if(leaf_saver[i].counter==0):
        S = "[{},{},{}]   {}\n".format(leaf_saver[i].redSum,leaf_saver[i].greenSum,leaf_saver[i].blueSum,0)
    else:
        red = leaf_saver[i].redSum/leaf_saver[i].counter
        green = leaf_saver[i].greenSum/leaf_saver[i].counter
        blue = leaf_saver[i].blueSum/leaf_saver[i].counter
        S = "[{:.2f},{:.2f},{:.2f}]   {}\n".format(red,green,blue,leaf_saver[i].counter)
    f.write(S)
f.close()

In [7]:
data2 = np.array(Image.open('task2.jpg'))

In [12]:
for i in range(data2.shape[0]):
    for j in range(data2.shape[1]):
        node = Octree.root
        index = 0
        while(node.leaf == False):
            pixel = data2[i][j]
            red = Dec2Bi(pixel[0])[index]
            green = Dec2Bi(pixel[1])[index]
            blue = Dec2Bi(pixel[2])[index]
            loc = red*4+green*2+blue
            node = node.child_node[loc]
            index += 1
        if(node.counter == 0):
            R = node.redSum
            G = node.greenSum
            B = node.blueSum
        else:
            R = node.redSum/node.counter
            G = node.greenSum/node.counter
            B = node.blueSum/node.counter
        a = [int(R),int(G),int(B)]
        data2[i][j]=np.array(a)

In [17]:
img = Image.fromarray(data2)

In [16]:
img.save('Desktop/task2_D.jpg')